In [1]:
import sys
import os
myDir = os.getcwd()
parentDir = os.path.abspath(os.path.join(myDir, os.pardir))
sys.path.append(parentDir)

from pathlib import Path
path = Path(parentDir)
a=str(path.parent.absolute())

sys.path.append(a)

In [2]:
from MetaX.utils.taxaFuncAnalyzer import TaxaFuncAnalyzer
from MetaX.utils.taxaFuncPloter.basic_plot import BasicPlot


In [3]:
df_path = './example_data/SW_TaxaFunc.tsv'
meta_path = './example_data/SW_Meta.tsv'
sw = TaxaFuncAnalyzer(df_path, meta_path)

original df shape: (3999, 229)
after remove all zero row: (3999, 229)


In [4]:
sw.set_func('Description')
sw.set_multi_tables(level='s')

sw.set_group('Person')

batch_list is not set, Batch effect removal did not perform.
transform_method is not set, data transform did not perform.
normalize_method is not set, data normalization did not perform.
Original data shape: (3999, 229)
Starting to set Function table...
Function number: 830
Starting to set Taxa table...
Taxa number: 340
Taxa-Function number: 1272
group is set to Person
 {'V1', 'V2', 'V3', 'V4', 'V5'}


In [10]:
sw.func_df

,V1_01,V1_02,V1_03,V1_04,V1_05,V1_06,V1_07,V1_08,V1_09,V1_10,...,V5_29,V5_30,V5_31,V5_32,V5_33,V5_34,V5_35,V5_36,V5_37,V5_38
Description,,,,,,,,,,,,,,,,,,,,,
'glutamate synthase,0,0,0,0,0,0,0,0,0,0,...,0,995468,0,0,784835,1994196,1210561,2690660,2471832,388420
"2-hydroxyglutaryl-CoA dehydratase, D-component",663707,1020739,277743,80781,319969,536173,148394,1134756,119011,1013845,...,0,0,0,124831,0,241352,173366,0,0,0
"2-oxoacid acceptor oxidoreductase, alpha subunit",220510,251580,726890,164401,0,522650,413416,0,1090034,937169,...,381664,507241,200403,574218,703480,626598,803540,605593,1025796,903907
3'-to-5' exoribonuclease specific for small oligoribonucleotides,0,0,0,0,0,0,108026,0,0,254582,...,914736,1470116,766459,493726,0,0,0,0,0,0
3-Oxoacyl-[acyl-carrier-protein (ACP)] synthase III,736277,1120188,1678680,720611,1755009,827877,516740,729217,869831,758829,...,3298006,808629,1969936,0,739544,2282781,2793189,3355780,1789019,2202588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
thus facilitating recognition of the initiation point. It is needed to translate mRNA with a short Shine-Dalgarno (SD) purine-rich sequence,8724998,8623683,14550662,8410821,11258692,7672131,11314813,8950486,8636976,9712674,...,5154606,8209722,12538977,4989211,4589739,6681835,8611700,2954839,16778048,4461302
transcriptional regulator,2587019,2114605,2456893,2638791,1831952,5169727,3571152,7308014,3684445,6428215,...,0,0,0,0,0,0,0,0,0,0
translation elongation,1600329,2171583,2463681,2188611,1032612,1659640,1369216,1355185,1631697,2489780,...,479525,1649074,1032024,889564,949668,124432,475618,1306939,1291938,1423492


In [14]:
l1 = sw.taxa_func_df.index.to_list()
taxa_func_list = []
for i in (l1):
    taxa = i[0]
    func = i[1]
    taxa_func_list.append(taxa + f' <' + func + '>') 
list(set(taxa_func_list))

['d__Bacteria|p__Bacteroidota|c__Bacteroidia|o__Bacteroidales|f__Tannerellaceae|g__Parabacteroides|s__Parabacteroides distasonis <thus facilitating recognition of the initiation point. It is needed to translate mRNA with a short Shine-Dalgarno (SD) purine-rich sequence>',
 'd__Bacteria|p__Firmicutes_A|c__Clostridia|o__Lachnospirales|f__Lachnospiraceae|g__Anaerostipes|s__Anaerostipes hadrus <dextransucrase activity>',
 'd__Bacteria|p__Bacteroidota|c__Bacteroidia|o__Bacteroidales|f__Rikenellaceae|g__Alistipes|s__Alistipes communis <Methylmalonyl-CoA mutase>',
 'd__Bacteria|p__Firmicutes_A|c__Clostridia|o__Peptostreptococcales|f__Anaerovoracaceae|g__UBA1191|s__UBA1191 sp900543485 <Glycine/sarcosine/betaine reductase component B subunits>',
 'd__Bacteria|p__Proteobacteria|c__Gammaproteobacteria|o__Enterobacterales|f__Enterobacteriaceae|g__Escherichia|s__Escherichia coli_D <Forms part of the polypeptide exit tunnel>',
 'd__Bacteria|p__Proteobacteria|c__Gammaproteobacteria|o__Burkholderiales